In [ ]:
#20200427
#iron crystal 
#train pix2pix 
%run ./scripts/train_pix2pix.py   --name iron_label2real_pix2pix_real_35 \
--gpu_ids 0,1 --batch_size 8 --checkpoints_dir './ckpts/aug_models' \
--dataroot './datasets/' --phase 'train' --label_dir 'real_labels' --image_dir 'real_images' \
--record_txt './datasets/train/records_txt/real_35.txt' \
--display_port 6006 --display_env 'real_35'

----------------- Options ---------------
               batch_size: 8                             	[default: 1]
                    beta1: 0.5                           
          checkpoints_dir: ./ckpts/aug_models            
           continue_train: False                         
                 dataroot: ./datasets/                   	[default: ./datasets]
             dataset_mode: pix2pix                       
              display_env: real_35                       	[default: main]
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
             display_port: 6006                          	[default: 8097]
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest                        
              epoch_count: 1                             
                 fine

initialize network with normal
initialize network with normal
model [Pix2PixModel] was created
---------- Networks initialized -------------
[Network G] Total number of parameters : 12.983 M
[Network D] Total number of parameters : 2.765 M
-----------------------------------------------
create images directory ./ckpts/aug_models/iron_label2real_pix2pix_real_35/images...
(epoch: 1, iters: 200, time: 0.026, data: 0.384) G_GAN: 3.3271 G_L1: 72.7101 D_real: 0.0468 D_fake: 0.0424 
(epoch: 1, iters: 400, time: 0.073, data: 0.004) G_GAN: 2.7530 G_L1: 23.0155 D_real: 0.1394 D_fake: 0.0473 
(epoch: 1, iters: 600, time: 0.026, data: 0.005) G_GAN: 0.9521 G_L1: 9.4394 D_real: 0.5246 D_fake: 0.5218 
(epoch: 1, iters: 800, time: 0.066, data: 0.005) G_GAN: 1.9466 G_L1: 9.2431 D_real: 0.1227 D_fake: 0.2052 
End of epoch 1 / 200 	 Time Taken: 29 sec
learning rate = 0.0002000
(epoch: 2, iters: 16, time: 0.026, data: 0.004) G_GAN: 2.6991 G_L1: 9.5507 D_real: 0.0466 D_fake: 0.0924 
(epoch: 2, iters: 216, 

(epoch: 14, iters: 8, time: 0.072, data: 0.004) G_GAN: 0.9837 G_L1: 10.9584 D_real: 0.7047 D_fake: 0.3222 
(epoch: 14, iters: 208, time: 0.027, data: 0.006) G_GAN: 2.0271 G_L1: 9.6444 D_real: 0.4563 D_fake: 0.9911 
(epoch: 14, iters: 408, time: 0.069, data: 0.003) G_GAN: 0.7964 G_L1: 9.8450 D_real: 1.0621 D_fake: 0.4058 
(epoch: 14, iters: 608, time: 0.027, data: 0.004) G_GAN: 0.9168 G_L1: 8.8992 D_real: 0.5684 D_fake: 0.5890 
(epoch: 14, iters: 808, time: 0.069, data: 0.004) G_GAN: 1.1094 G_L1: 10.0146 D_real: 0.6937 D_fake: 0.3482 
End of epoch 14 / 200 	 Time Taken: 23 sec
learning rate = 0.0002000
(epoch: 15, iters: 24, time: 0.027, data: 0.003) G_GAN: 1.3645 G_L1: 10.6171 D_real: 0.2834 D_fake: 0.6268 
(epoch: 15, iters: 224, time: 0.066, data: 0.002) G_GAN: 1.1324 G_L1: 9.7413 D_real: 0.7204 D_fake: 0.4199 
(epoch: 15, iters: 424, time: 0.027, data: 0.003) G_GAN: 0.7507 G_L1: 9.9614 D_real: 0.5890 D_fake: 0.6752 
(epoch: 15, iters: 624, time: 0.069, data: 0.003) G_GAN: 1.2931 G_L

(epoch: 27, iters: 216, time: 0.063, data: 0.002) G_GAN: 1.3536 G_L1: 10.5329 D_real: 0.3598 D_fake: 0.5896 
(epoch: 27, iters: 416, time: 0.103, data: 0.003) G_GAN: 2.2993 G_L1: 10.8577 D_real: 0.1786 D_fake: 1.0194 
(epoch: 27, iters: 616, time: 0.053, data: 0.005) G_GAN: 0.7161 G_L1: 9.5372 D_real: 0.8975 D_fake: 0.6530 
(epoch: 27, iters: 816, time: 0.105, data: 0.004) G_GAN: 0.6858 G_L1: 10.2741 D_real: 0.5579 D_fake: 0.5790 
End of epoch 27 / 200 	 Time Taken: 42 sec
learning rate = 0.0002000
(epoch: 28, iters: 32, time: 0.059, data: 0.004) G_GAN: 1.4009 G_L1: 10.6634 D_real: 0.3606 D_fake: 0.4297 
(epoch: 28, iters: 232, time: 0.076, data: 0.005) G_GAN: 0.7612 G_L1: 10.1860 D_real: 0.5068 D_fake: 0.8872 
(epoch: 28, iters: 432, time: 0.058, data: 0.004) G_GAN: 0.8160 G_L1: 9.9357 D_real: 0.4573 D_fake: 0.7180 
(epoch: 28, iters: 632, time: 0.104, data: 0.003) G_GAN: 1.5361 G_L1: 10.0030 D_real: 0.3317 D_fake: 0.6728 
(epoch: 28, iters: 832, time: 0.059, data: 0.004) G_GAN: 0.893

(epoch: 40, iters: 624, time: 0.026, data: 0.006) G_GAN: 1.1175 G_L1: 9.6744 D_real: 0.4090 D_fake: 0.6131 
(epoch: 40, iters: 824, time: 0.091, data: 0.005) G_GAN: 1.4125 G_L1: 10.0146 D_real: 0.3920 D_fake: 0.2771 
saving the model at the end of epoch 40, iters 39360
End of epoch 40 / 200 	 Time Taken: 42 sec
learning rate = 0.0002000
(epoch: 41, iters: 40, time: 0.060, data: 0.004) G_GAN: 1.2038 G_L1: 9.4557 D_real: 0.4658 D_fake: 0.5005 
(epoch: 41, iters: 240, time: 0.090, data: 0.006) G_GAN: 0.7794 G_L1: 9.2662 D_real: 0.7212 D_fake: 0.3250 
(epoch: 41, iters: 440, time: 0.058, data: 0.003) G_GAN: 1.2451 G_L1: 10.3146 D_real: 0.4837 D_fake: 0.4998 
(epoch: 41, iters: 640, time: 0.101, data: 0.003) G_GAN: 1.0680 G_L1: 9.4342 D_real: 0.6423 D_fake: 0.3957 
saving the latest model (epoch 41, total_steps 40000)
(epoch: 41, iters: 840, time: 0.028, data: 0.003) G_GAN: 1.7365 G_L1: 9.3361 D_real: 0.2436 D_fake: 0.6736 
End of epoch 41 / 200 	 Time Taken: 42 sec
learning rate = 0.000200

In [ ]:
#pix2pix
#generate synthetic data
%run ./scripts/generate_samples.py  --name iron_label2real_pix2pix_real_35  --num_test 28800 \
--gpu_ids 0 --batch_size 1 --checkpoints_dir './ckpts/aug_models/' \
--dataroot './datasets/' --phase 'train' --label_dir 'syn_labels' \
--results_dir './datasets/train/'  --epoch 'latest'

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./ckpts/aug_models/           	[default: ./ckpts/aug_models]
                 dataroot: ./datasets/                   	[default: ./datasets]
             dataset_mode: pix2pix                       
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                 fineSize: 400                           
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 1                             
                  isTrain: False                         	[default: None]
                label_dir: syn_labels                    	[default: synthtic_l

processing (0415)-th image... ./datasets/train/syn_labels/10370.png
processing (0420)-th image... ./datasets/train/syn_labels/10375.png
processing (0425)-th image... ./datasets/train/syn_labels/1038.png
processing (0430)-th image... ./datasets/train/syn_labels/10384.png
processing (0435)-th image... ./datasets/train/syn_labels/10389.png
processing (0440)-th image... ./datasets/train/syn_labels/10393.png
processing (0445)-th image... ./datasets/train/syn_labels/10398.png
processing (0450)-th image... ./datasets/train/syn_labels/10401.png
processing (0455)-th image... ./datasets/train/syn_labels/10406.png
processing (0460)-th image... ./datasets/train/syn_labels/10410.png
processing (0465)-th image... ./datasets/train/syn_labels/10415.png
processing (0470)-th image... ./datasets/train/syn_labels/1042.png
processing (0475)-th image... ./datasets/train/syn_labels/10424.png
processing (0480)-th image... ./datasets/train/syn_labels/10429.png
processing (0485)-th image... ./datasets/train/syn

processing (1020)-th image... ./datasets/train/syn_labels/10915.png
processing (1025)-th image... ./datasets/train/syn_labels/1092.png
processing (1030)-th image... ./datasets/train/syn_labels/10924.png
processing (1035)-th image... ./datasets/train/syn_labels/10929.png
processing (1040)-th image... ./datasets/train/syn_labels/10933.png
processing (1045)-th image... ./datasets/train/syn_labels/10938.png
processing (1050)-th image... ./datasets/train/syn_labels/10942.png
processing (1055)-th image... ./datasets/train/syn_labels/10947.png
processing (1060)-th image... ./datasets/train/syn_labels/10951.png
processing (1065)-th image... ./datasets/train/syn_labels/10956.png
processing (1070)-th image... ./datasets/train/syn_labels/10960.png
processing (1075)-th image... ./datasets/train/syn_labels/10965.png
processing (1080)-th image... ./datasets/train/syn_labels/1097.png
processing (1085)-th image... ./datasets/train/syn_labels/10974.png
processing (1090)-th image... ./datasets/train/syn

processing (1625)-th image... ./datasets/train/syn_labels/1146.png
processing (1630)-th image... ./datasets/train/syn_labels/11464.png
processing (1635)-th image... ./datasets/train/syn_labels/11469.png
processing (1640)-th image... ./datasets/train/syn_labels/11473.png
processing (1645)-th image... ./datasets/train/syn_labels/11478.png
processing (1650)-th image... ./datasets/train/syn_labels/11482.png
processing (1655)-th image... ./datasets/train/syn_labels/11487.png
processing (1660)-th image... ./datasets/train/syn_labels/11491.png
processing (1665)-th image... ./datasets/train/syn_labels/11496.png
processing (1670)-th image... ./datasets/train/syn_labels/1150.png
processing (1675)-th image... ./datasets/train/syn_labels/11504.png
processing (1680)-th image... ./datasets/train/syn_labels/11509.png
processing (1685)-th image... ./datasets/train/syn_labels/11513.png
processing (1690)-th image... ./datasets/train/syn_labels/11518.png
processing (1695)-th image... ./datasets/train/syn

processing (2235)-th image... ./datasets/train/syn_labels/12008.png
processing (2240)-th image... ./datasets/train/syn_labels/12012.png
processing (2245)-th image... ./datasets/train/syn_labels/12017.png
processing (2250)-th image... ./datasets/train/syn_labels/12021.png
processing (2255)-th image... ./datasets/train/syn_labels/12026.png
processing (2260)-th image... ./datasets/train/syn_labels/12030.png
processing (2265)-th image... ./datasets/train/syn_labels/12035.png
processing (2270)-th image... ./datasets/train/syn_labels/1204.png
processing (2275)-th image... ./datasets/train/syn_labels/12044.png
processing (2280)-th image... ./datasets/train/syn_labels/12049.png
processing (2285)-th image... ./datasets/train/syn_labels/12053.png
processing (2290)-th image... ./datasets/train/syn_labels/12058.png
processing (2295)-th image... ./datasets/train/syn_labels/12062.png
processing (2300)-th image... ./datasets/train/syn_labels/12067.png
processing (2305)-th image... ./datasets/train/sy

...

processing (18500)-th image... ./datasets/train/syn_labels/26648.png
processing (18505)-th image... ./datasets/train/syn_labels/26652.png
processing (18510)-th image... ./datasets/train/syn_labels/26657.png
processing (18515)-th image... ./datasets/train/syn_labels/26661.png
processing (18520)-th image... ./datasets/train/syn_labels/26666.png
processing (18525)-th image... ./datasets/train/syn_labels/26670.png
processing (18530)-th image... ./datasets/train/syn_labels/26675.png
processing (18535)-th image... ./datasets/train/syn_labels/2668.png
processing (18540)-th image... ./datasets/train/syn_labels/26684.png
processing (18545)-th image... ./datasets/train/syn_labels/26689.png
processing (18550)-th image... ./datasets/train/syn_labels/26693.png
processing (18555)-th image... ./datasets/train/syn_labels/26698.png
processing (18560)-th image... ./datasets/train/syn_labels/26701.png
processing (18565)-th image... ./datasets/train/syn_labels/26706.png
processing (18570)-th image... ./da

processing (19100)-th image... ./datasets/train/syn_labels/27188.png
processing (19105)-th image... ./datasets/train/syn_labels/27192.png
processing (19110)-th image... ./datasets/train/syn_labels/27197.png
processing (19115)-th image... ./datasets/train/syn_labels/27200.png
processing (19120)-th image... ./datasets/train/syn_labels/27205.png
processing (19125)-th image... ./datasets/train/syn_labels/2721.png
processing (19130)-th image... ./datasets/train/syn_labels/27214.png
processing (19135)-th image... ./datasets/train/syn_labels/27219.png
processing (19140)-th image... ./datasets/train/syn_labels/27223.png
processing (19145)-th image... ./datasets/train/syn_labels/27228.png
processing (19150)-th image... ./datasets/train/syn_labels/27232.png
processing (19155)-th image... ./datasets/train/syn_labels/27237.png
processing (19160)-th image... ./datasets/train/syn_labels/27241.png
processing (19165)-th image... ./datasets/train/syn_labels/27246.png
processing (19170)-th image... ./da

processing (19695)-th image... ./datasets/train/syn_labels/27723.png
processing (19700)-th image... ./datasets/train/syn_labels/27728.png
processing (19705)-th image... ./datasets/train/syn_labels/27732.png
processing (19710)-th image... ./datasets/train/syn_labels/27737.png
processing (19715)-th image... ./datasets/train/syn_labels/27741.png
processing (19720)-th image... ./datasets/train/syn_labels/27746.png
processing (19725)-th image... ./datasets/train/syn_labels/27750.png
processing (19730)-th image... ./datasets/train/syn_labels/27755.png
processing (19735)-th image... ./datasets/train/syn_labels/2776.png
processing (19740)-th image... ./datasets/train/syn_labels/27764.png
processing (19745)-th image... ./datasets/train/syn_labels/27769.png
processing (19750)-th image... ./datasets/train/syn_labels/27773.png
processing (19755)-th image... ./datasets/train/syn_labels/27778.png
processing (19760)-th image... ./datasets/train/syn_labels/27782.png
processing (19765)-th image... ./da

processing (20295)-th image... ./datasets/train/syn_labels/28263.png
processing (20300)-th image... ./datasets/train/syn_labels/28268.png
processing (20305)-th image... ./datasets/train/syn_labels/28272.png
processing (20310)-th image... ./datasets/train/syn_labels/28277.png
processing (20315)-th image... ./datasets/train/syn_labels/28281.png
processing (20320)-th image... ./datasets/train/syn_labels/28286.png
processing (20325)-th image... ./datasets/train/syn_labels/28290.png
processing (20330)-th image... ./datasets/train/syn_labels/28295.png
processing (20335)-th image... ./datasets/train/syn_labels/283.png
processing (20340)-th image... ./datasets/train/syn_labels/28303.png
processing (20345)-th image... ./datasets/train/syn_labels/28308.png
processing (20350)-th image... ./datasets/train/syn_labels/28312.png
processing (20355)-th image... ./datasets/train/syn_labels/28317.png
processing (20360)-th image... ./datasets/train/syn_labels/28321.png
processing (20365)-th image... ./dat

processing (20890)-th image... ./datasets/train/syn_labels/288.png
processing (20895)-th image... ./datasets/train/syn_labels/2884.png
processing (20900)-th image... ./datasets/train/syn_labels/2889.png
processing (20905)-th image... ./datasets/train/syn_labels/2893.png
processing (20910)-th image... ./datasets/train/syn_labels/2898.png
processing (20915)-th image... ./datasets/train/syn_labels/2901.png
processing (20920)-th image... ./datasets/train/syn_labels/2906.png
processing (20925)-th image... ./datasets/train/syn_labels/2910.png
processing (20930)-th image... ./datasets/train/syn_labels/2915.png
processing (20935)-th image... ./datasets/train/syn_labels/292.png
processing (20940)-th image... ./datasets/train/syn_labels/2924.png
processing (20945)-th image... ./datasets/train/syn_labels/2929.png
processing (20950)-th image... ./datasets/train/syn_labels/2933.png
processing (20955)-th image... ./datasets/train/syn_labels/2938.png
processing (20960)-th image... ./datasets/train/sy

In [1]:
#our method
#mix train unet
%run ./scripts/train.py --net unet --train_type 'mix' --gpu_ids 2,3 \
--dataroot './datasets/' --batch_size 8 \
--train_img_dir_real 'train/real_images' \
--train_label_dir_real 'train/real_labels' \
--train_img_list_real './datasets/train/records_txt/real_10.txt' \
--train_img_dir_syn 'train/iron_label2real_pix2pix_real_10/latest/images' \
--train_label_dir_syn 'train/iron_label2real_pix2pix_real_10/latest/labels' \
--checkpoints_dir './ckpts/seg_models' \
--name 'model_real_10_mix_syn' \
--display_env 'model_real_10_mix_syn' \
--display_port 6006 

---------Options---------
               batch_size: 8                             	[default: 1]
          checkpoints_dir: ./ckpts/seg_models            	[default: ./model_checkpoints]
           continue_train: False                         
                 dataroot: ./datasets/                   	[default: ./datasets]
             dataset_mode: aligned                       
              display_env: model_real_10_mix_syn         	[default: main]
               display_id: 1                             
            display_ncols: 4                             
             display_port: 6006                          	[default: 8097]
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: best_eval                     
              epoch_count: 1                             
                  gpu_ids: [2, 3]                        	[default: 0,1]
                init_gain: 1.0                   

initialize network with xavier_uniform
model [UnetModel] was created
---------- Networks initialized -------------
[Network ] Total number of parameters : 31.042 M
-----------------------------------------------
create images directory ./ckpts/seg_models/model_real_10_mix_syn/images...
Training begin...total epochs:5
saving the latest model (epoch 1, total_steps 2000)
saving the latest model (epoch 1, total_steps 4000)
saving the latest model (epoch 1, total_steps 6000)
saving the eval best model (epoch 1, total_steps 7200)
(epoch: 1, iters: 1800, time: 2.083, data: 0.287) train_loss: 0.0818 eval_loss: 0.2289 
saving the model (epoch_1_1800)
saving the latest model (epoch 1, total_steps 8000)
saving the latest model (epoch 1, total_steps 10000)
saving the latest model (epoch 1, total_steps 12000)
saving the latest model (epoch 1, total_steps 14000)
(epoch: 1, iters: 3600, time: 2.220, data: 0.010) train_loss: 0.0509 eval_loss: 0.2538 
saving the model (epoch_1_3600)
saving the latest m

In [1]:
#test our method
%run ./scripts/test.py --net unet --name 'model_real_10_mix_syn'\
--checkpoints_dir './ckpts/seg_models/' \
--dataroot './datasets/test/' \
--test_img_dir 'real_images' \
--test_label_dir 'real_labels' \
--epoch 'epoch_5_7200' --results_dir './results/' --gpu_ids 0

---------Options---------
               batch_size: 1                             
                 boundary: 255                           
          checkpoints_dir: ./ckpts/seg_models/           	[default: ./model_checkpoints]
                 dataroot: ./datasets/test/              	[default: ./datasets]
             dataset_mode: aligned                       
                    epoch: epoch_5_7200                  	[default: best_eval]
                  gpu_ids: [0]                           	[default: 0,1]
                init_gain: 1.0                           
                init_type: xavier_uniform                
                 input_nc: 1                             
                  isTrain: False                         	[default: None]
                load_iter: 0                             	[default: 0]
                loss_type: CrossEntropyLoss              
              loss_weight: [1, 20]                       	[default: 1:20]
         max_dataset_size: i

0039: 067_6_3 	map_score: 0.55498	m_iou: 0.79189	vi: 0.36758	ri: 0.98915	adjust_ri: 0.87059	merger_error: 0.18153	split_error: 0.18605	 cost: 0.0636
0040: 063_7_2 	map_score: 0.58347	m_iou: 0.74091	vi: 0.33905	ri: 0.98974	adjust_ri: 0.88713	merger_error: 0.20016	split_error: 0.13889	 cost: 0.0630
0041: 019_1_3 	map_score: 0.61092	m_iou: 0.78607	vi: 0.33198	ri: 0.98986	adjust_ri: 0.85415	merger_error: 0.23135	split_error: 0.10063	 cost: 0.0778
0042: 075_7_4 	map_score: 0.37890	m_iou: 0.74211	vi: 0.57914	ri: 0.98925	adjust_ri: 0.83184	merger_error: 0.39389	split_error: 0.18525	 cost: 0.0968
0043: 031_1_2 	map_score: 0.53289	m_iou: 0.76406	vi: 0.42389	ri: 0.98609	adjust_ri: 0.78560	merger_error: 0.32171	split_error: 0.10217	 cost: 0.0775
0044: 063_5_3 	map_score: 0.52495	m_iou: 0.65704	vi: 0.40068	ri: 0.98990	adjust_ri: 0.86103	merger_error: 0.09739	split_error: 0.30330	 cost: 0.0719
0045: 068_4_1 	map_score: 0.57376	m_iou: 0.72595	vi: 0.30043	ri: 0.99260	adjust_ri: 0.87892	merger_error: 

0095: 020_7_2 	map_score: 0.52363	m_iou: 0.77198	vi: 0.88118	ri: 0.95135	adjust_ri: 0.53416	merger_error: 0.67707	split_error: 0.20412	 cost: 0.0818
0096: 038_7_2 	map_score: 0.57115	m_iou: 0.76678	vi: 0.31763	ri: 0.99067	adjust_ri: 0.88077	merger_error: 0.20888	split_error: 0.10875	 cost: 0.0780
0097: 040_3_2 	map_score: 0.46951	m_iou: 0.66039	vi: 0.44282	ri: 0.98869	adjust_ri: 0.86325	merger_error: 0.22883	split_error: 0.21399	 cost: 0.0726
0098: 038_5_2 	map_score: 0.49990	m_iou: 0.74101	vi: 0.48856	ri: 0.98882	adjust_ri: 0.80613	merger_error: 0.23308	split_error: 0.25548	 cost: 0.0749
0099: 044_6_2 	map_score: 0.46024	m_iou: 0.66002	vi: 0.29423	ri: 0.99383	adjust_ri: 0.90369	merger_error: 0.11425	split_error: 0.17998	 cost: 0.0872
0100: 016_4_4 	map_score: 0.55305	m_iou: 0.78840	vi: 0.54531	ri: 0.98327	adjust_ri: 0.78173	merger_error: 0.37833	split_error: 0.16698	 cost: 0.0768
0101: 064_1_1 	map_score: 0.31758	m_iou: 0.75163	vi: 0.24039	ri: 0.99447	adjust_ri: 0.91913	merger_error: 

0151: 082_7_3 	map_score: 0.59395	m_iou: 0.77071	vi: 0.31519	ri: 0.99155	adjust_ri: 0.87307	merger_error: 0.17076	split_error: 0.14444	 cost: 0.0774
0152: 020_5_4 	map_score: 0.53861	m_iou: 0.74890	vi: 0.49877	ri: 0.97586	adjust_ri: 0.73092	merger_error: 0.37949	split_error: 0.11928	 cost: 0.0705
0153: 016_1_1 	map_score: 0.55302	m_iou: 0.76574	vi: 0.40699	ri: 0.98973	adjust_ri: 0.82763	merger_error: 0.32436	split_error: 0.08263	 cost: 0.0824
0154: 001_1_4 	map_score: 0.74703	m_iou: 0.88450	vi: 0.21785	ri: 0.99207	adjust_ri: 0.90953	merger_error: 0.15876	split_error: 0.05909	 cost: 0.0723
0155: 042_7_4 	map_score: 0.54959	m_iou: 0.68904	vi: 0.39562	ri: 0.99131	adjust_ri: 0.87548	merger_error: 0.09937	split_error: 0.29624	 cost: 0.0781
0156: 031_3_4 	map_score: 0.59227	m_iou: 0.77467	vi: 0.40295	ri: 0.98416	adjust_ri: 0.78592	merger_error: 0.28172	split_error: 0.12123	 cost: 0.0807
0157: 038_2_2 	map_score: 0.49941	m_iou: 0.70738	vi: 0.50598	ri: 0.98424	adjust_ri: 0.78598	merger_error: 

0207: 044_6_1 	map_score: 0.43298	m_iou: 0.71155	vi: 0.59357	ri: 0.98103	adjust_ri: 0.75891	merger_error: 0.41420	split_error: 0.17937	 cost: 0.0732
0208: 038_3_4 	map_score: 0.54761	m_iou: 0.70754	vi: 0.40888	ri: 0.98959	adjust_ri: 0.84754	merger_error: 0.16657	split_error: 0.24231	 cost: 0.0828
0209: 029_6_2 	map_score: 0.50210	m_iou: 0.77289	vi: 0.42901	ri: 0.98652	adjust_ri: 0.82162	merger_error: 0.34321	split_error: 0.08579	 cost: 0.0752
0210: 055_1_3 	map_score: 0.71325	m_iou: 0.80803	vi: 0.17803	ri: 0.99547	adjust_ri: 0.93341	merger_error: 0.08340	split_error: 0.09463	 cost: 0.0790
0211: 058_1_2 	map_score: 0.47650	m_iou: 0.69620	vi: 0.50420	ri: 0.98401	adjust_ri: 0.78505	merger_error: 0.24552	split_error: 0.25868	 cost: 0.0796
0212: 055_3_4 	map_score: 0.65768	m_iou: 0.83635	vi: 0.30287	ri: 0.99092	adjust_ri: 0.85992	merger_error: 0.17946	split_error: 0.12341	 cost: 0.0779
0213: 083_2_2 	map_score: 0.60048	m_iou: 0.72891	vi: 0.31503	ri: 0.99324	adjust_ri: 0.89170	merger_error: 

...

0767: 070_2_3 	map_score: 0.57446	m_iou: 0.79135	vi: 0.44668	ri: 0.98770	adjust_ri: 0.82850	merger_error: 0.22488	split_error: 0.22180	 cost: 0.0731
0768: 055_7_4 	map_score: 0.57004	m_iou: 0.73177	vi: 0.27759	ri: 0.99356	adjust_ri: 0.91661	merger_error: 0.06048	split_error: 0.21711	 cost: 0.0761
0769: 076_3_4 	map_score: 0.55710	m_iou: 0.77375	vi: 0.42665	ri: 0.98506	adjust_ri: 0.81146	merger_error: 0.32659	split_error: 0.10006	 cost: 0.0924
0770: 053_1_4 	map_score: 0.62685	m_iou: 0.87754	vi: 0.27341	ri: 0.99235	adjust_ri: 0.89992	merger_error: 0.21977	split_error: 0.05364	 cost: 0.0759
0771: 078_3_3 	map_score: 0.60412	m_iou: 0.78615	vi: 0.24594	ri: 0.99210	adjust_ri: 0.89719	merger_error: 0.09854	split_error: 0.14739	 cost: 0.0664
0772: 024_5_3 	map_score: 0.60985	m_iou: 0.85131	vi: 0.39145	ri: 0.98753	adjust_ri: 0.84060	merger_error: 0.33220	split_error: 0.05925	 cost: 0.0905
0773: 018_6_4 	map_score: 0.49462	m_iou: 0.77432	vi: 0.47222	ri: 0.98536	adjust_ri: 0.80799	merger_error: 

0823: 047_4_2 	map_score: 0.55844	m_iou: 0.80169	vi: 0.47230	ri: 0.98371	adjust_ri: 0.79590	merger_error: 0.34182	split_error: 0.13048	 cost: 0.0759
0824: 085_4_2 	map_score: 0.68485	m_iou: 0.81002	vi: 0.23627	ri: 0.99388	adjust_ri: 0.89929	merger_error: 0.06941	split_error: 0.16686	 cost: 0.0793
0825: 046_1_2 	map_score: 0.52675	m_iou: 0.76706	vi: 0.43411	ri: 0.98695	adjust_ri: 0.81198	merger_error: 0.33862	split_error: 0.09549	 cost: 0.0725
0826: 076_4_4 	map_score: 0.37738	m_iou: 0.81971	vi: 0.52366	ri: 0.97237	adjust_ri: 0.73267	merger_error: 0.45510	split_error: 0.06856	 cost: 0.0694
0827: 071_2_3 	map_score: 0.61691	m_iou: 0.77806	vi: 0.45322	ri: 0.98743	adjust_ri: 0.82434	merger_error: 0.22139	split_error: 0.23183	 cost: 0.0802
0828: 072_1_3 	map_score: 0.57710	m_iou: 0.72448	vi: 0.40959	ri: 0.98584	adjust_ri: 0.81594	merger_error: 0.21168	split_error: 0.19791	 cost: 0.0739
0829: 025_7_4 	map_score: 0.58871	m_iou: 0.80266	vi: 0.29680	ri: 0.99161	adjust_ri: 0.90776	merger_error: 

0879: 056_7_2 	map_score: 0.46204	m_iou: 0.73368	vi: 0.54565	ri: 0.98082	adjust_ri: 0.78939	merger_error: 0.35328	split_error: 0.19238	 cost: 0.0716
0880: 073_3_4 	map_score: 0.63797	m_iou: 0.83732	vi: 0.36966	ri: 0.98738	adjust_ri: 0.83888	merger_error: 0.28133	split_error: 0.08833	 cost: 0.0663
0881: 007_5_1 	map_score: 0.46638	m_iou: 0.65666	vi: 0.47282	ri: 0.98802	adjust_ri: 0.84469	merger_error: 0.27024	split_error: 0.20257	 cost: 0.0796
0882: 034_2_2 	map_score: 0.61301	m_iou: 0.75521	vi: 0.22941	ri: 0.99423	adjust_ri: 0.91878	merger_error: 0.05711	split_error: 0.17230	 cost: 0.0724
0883: 062_4_3 	map_score: 0.65292	m_iou: 0.78487	vi: 0.21607	ri: 0.99420	adjust_ri: 0.92924	merger_error: 0.13591	split_error: 0.08016	 cost: 0.0794
0884: 044_1_4 	map_score: 0.54605	m_iou: 0.71754	vi: 0.22050	ri: 0.99519	adjust_ri: 0.93815	merger_error: 0.09528	split_error: 0.12522	 cost: 0.0795
0885: 014_4_2 	map_score: 0.63613	m_iou: 0.81892	vi: 0.28309	ri: 0.99257	adjust_ri: 0.88714	merger_error: 

0935: 015_3_4 	map_score: 0.63485	m_iou: 0.83132	vi: 0.33350	ri: 0.99000	adjust_ri: 0.86271	merger_error: 0.22368	split_error: 0.10982	 cost: 0.0765
0936: 003_5_2 	map_score: 0.52097	m_iou: 0.72198	vi: 0.53481	ri: 0.98392	adjust_ri: 0.79538	merger_error: 0.32262	split_error: 0.21219	 cost: 0.0690
0937: 036_6_1 	map_score: 0.54065	m_iou: 0.77981	vi: 0.47162	ri: 0.98422	adjust_ri: 0.79734	merger_error: 0.32825	split_error: 0.14337	 cost: 0.0712
0938: 062_7_3 	map_score: 0.55085	m_iou: 0.72929	vi: 0.40211	ri: 0.98870	adjust_ri: 0.84176	merger_error: 0.21760	split_error: 0.18451	 cost: 0.0803
0939: 035_3_3 	map_score: 0.57845	m_iou: 0.74538	vi: 0.35587	ri: 0.99166	adjust_ri: 0.86996	merger_error: 0.16320	split_error: 0.19267	 cost: 0.0835
0940: 005_1_4 	map_score: 0.51667	m_iou: 0.72155	vi: 0.51895	ri: 0.98601	adjust_ri: 0.82644	merger_error: 0.20407	split_error: 0.31487	 cost: 0.0734
0941: 037_7_4 	map_score: 0.32834	m_iou: 0.57184	vi: 0.78952	ri: 0.98087	adjust_ri: 0.76410	merger_error: 

0991: 004_5_3 	map_score: 0.60018	m_iou: 0.78261	vi: 0.36944	ri: 0.99059	adjust_ri: 0.85397	merger_error: 0.26215	split_error: 0.10729	 cost: 0.0737
0992: 064_2_1 	map_score: 0.46459	m_iou: 0.74973	vi: 0.35962	ri: 0.99051	adjust_ri: 0.87645	merger_error: 0.20650	split_error: 0.15312	 cost: 0.0709
0993: 057_6_1 	map_score: 0.62846	m_iou: 0.82920	vi: 0.25525	ri: 0.99361	adjust_ri: 0.90554	merger_error: 0.14346	split_error: 0.11180	 cost: 0.0726
0994: 072_3_1 	map_score: 0.47182	m_iou: 0.66431	vi: 0.46139	ri: 0.98757	adjust_ri: 0.82402	merger_error: 0.11430	split_error: 0.34709	 cost: 0.0935
0995: 002_6_3 	map_score: 0.67528	m_iou: 0.80460	vi: 0.24191	ri: 0.99108	adjust_ri: 0.87851	merger_error: 0.19159	split_error: 0.05032	 cost: 0.0870
0996: 078_7_2 	map_score: 0.70558	m_iou: 0.86713	vi: 0.23075	ri: 0.99468	adjust_ri: 0.92023	merger_error: 0.14200	split_error: 0.08875	 cost: 0.0738
0997: 008_4_4 	map_score: 0.50873	m_iou: 0.75519	vi: 0.40069	ri: 0.98828	adjust_ri: 0.83642	merger_error: 

In [ ]:
#origin train unet
%run ./scripts/train.py --net unet --train_type 'origin' --batch_size 8 --gpu_ids 0,1 \
--dataroot './datasets/' \
--train_img_dir_real 'train/real_images' \
--train_label_dir_real 'train/real_labels' \
--train_img_list_real './datasets/train/records_txt/real_35.txt' \
--val_img_dir 'val/real_images' \
--val_label_dir 'val/real_labels' \
--checkpoints_dir './ckpts/seg_models' \
--name 'model_real_35' \
--display_env 'model_real_35' \
--display_port 6006 

In [ ]:
#test origin unet
%run ./scripts/test.py --net unet --name 'model_real_5'\
--checkpoints_dir './ckpts/seg_models/' \
--dataroot './datasets/test/' \
--test_img_dir 'real_images' \
--test_label_dir 'real_labels' \
--results_dir './results/' --gpu_ids 0

In [ ]:
#ternausnet
#pretrained vgg11
%run ./scripts/train.py --net unet11 --train_type 'origin' --batch_size 8 --gpu_ids 0,1 \
--dataroot './datasets/' \
--train_img_dir_real 'train/real_images' \
--train_label_dir_real 'train/real_labels' \
--train_img_list_real './datasets/train/records_txt/real_35.txt' \
--val_img_dir 'val/real_images' \
--val_label_dir 'val/real_labels' \
--checkpoints_dir './ckpts/seg_models' \
--name 'model_real_35_pretrained_vgg11' \
--display_env 'model_real_35_pretrained_vgg11' \
--display_port 6006 

In [ ]:
#test ternausnet with pretrained vgg11
%run ./scripts/test.py --net unet11 --name 'model_real_35_pretrained_vgg11'\
--checkpoints_dir './ckpts/seg_models/' \
--dataroot './datasets/test/' \
--test_img_dir 'real_images' \
--test_label_dir 'real_labels' \
--epoch 'best_val' --results_dir './results/' --gpu_ids 0

In [ ]:
#auto image segmentation by threshold, morphology seg and edge detection
#otsu|canny|watershed|kmeans...
%run ./scripts/auto_segmentations.py --images_path './datasets/test/real_images/' --labels_path './datasets/test/real_labels/' \
--results_dir './results/others' --method 'kmeans'